In [53]:
import cx_Oracle

In [54]:
import pandas as pd

In [55]:
# conn = cx_Oracle.connect('hr/hr@localhost:1521/xe')
# 내 컴퓨터
conn = cx_Oracle.connect('hr/hr@192.168.0.4:1521/xe')

# 팀원 컴퓨터
# conn = cx_Oracle.connect('hr/hr@192.168.0.18:1521/xe')
conn

<cx_Oracle.Connection to hr@192.168.0.4:1521/xe>

In [56]:
cur = conn.cursor()

In [57]:
my_sql="select * from employees"

In [58]:
cur.execute(my_sql)
cur

<cx_Oracle.Cursor on <cx_Oracle.Connection to hr@192.168.0.4:1521/xe>>

In [59]:
df = pd.DataFrame(cur)
df

,0,1,2,3,4,5,6,7,8,9,10
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000.0,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000.0,NaN,100.0,90.0
2,102,Lex,De Haan,LDEHAAN,515.123.4569,2001-01-13,AD_VP,17000.0,NaN,100.0,90.0
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,2006-01-03,IT_PROG,9000.0,NaN,102.0,60.0
4,104,Bruce,Ernst,BERNST,590.423.4568,2007-05-21,IT_PROG,6000.0,NaN,103.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...
102,202,Pat,Fay,PFAY,603.123.6666,2005-08-17,MK_REP,6000.0,NaN,201.0,20.0
103,203,Susan,Mavris,SMAVRIS,515.123.7777,2002-06-07,HR_REP,6500.0,NaN,101.0,40.0
104,204,Hermann,Baer,HBAER,515.123.8888,2002-06-07,PR_REP,10000.0,NaN,101.0,70.0
105,205,Shelley,Higgins,SHIGGINS,515.123.8080,2002-06-07,AC_MGR,12008.0,NaN,101.0,110.0


In [60]:
conn = cx_Oracle.connect('open_source/1111@192.168.0.4:1521/xe')
conn

<cx_Oracle.Connection to open_source@192.168.0.4:1521/xe>

In [61]:
cur = conn.cursor()

In [63]:
# my_sql="select * from musicdata"
my_sql="select * from weather"
# my_sql="describe weather"
cur.execute(my_sql)
df = pd.DataFrame(cur)
df

""


In [48]:
# conn.commit() : db 변경 쿼리 (insert, update) 쿼리 수행이후
cur.close(); 
conn.close();

# conn.close();

In [37]:
# 목적: 파이썬 코드를 활용하여 OpenAPI 호출 자동화
# Step1: OpenAPI를 제공하는 사이트에서 제공하는 샘플 프로그램을 확보한다.

import requests
import time
import json
import pandas as pd
# access_key =
access_key='bgOnt78reFNsTUJuAwlI30JDObTxX6hbJCxyApJCtuf3xjJZJ/mOs8Vhg3GZAsLc1fXTkQ9sjq0mTEupWDdyyA=='
# 가산동 격자 x, 격자 y 정보
nx = 58
ny = 125
def get_request_url():
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
    params = {'serviceKey': access_key , 'numOfRows' : 10, 'pageNo' : 1,
        'dataType': 'JSON','base_date':base_date,'base_time':day_time,'nx':nx,'ny':ny
    }
    response = requests.get(url, params=params)
    # response.content # => response.content는 한글이 인코딩된 형식이므로
    #                       response.text 를 응답받기로함
    return response.text

def preprocess_df(df):

    df.insert(0,'date_time', df['baseDate']+df['baseTime'])

    p_df = pd.pivot(df, index='date_time', columns=['category'],values='obsrValue') # 하나의 행으로 데이터를 피봇
    nx = df.loc[0,'nx']
    ny = df.loc[0,'ny']
    date_time = df.loc[0,'baseDate'] + ' ' + df.loc[0,'baseTime']
    p_df.insert(0,'date_time',[date_time])
    p_df.insert(1,'nx',[nx])
    p_df.insert(2,'ny',[ny])

    p_df.rename(columns={
        'date_time':'DATE_TIME', 'nx':'NX', 'ny': 'NY', 'PTY':'강수형태',
        'REH':'습도','RN1':'시간1_강수량','T1H':'기온',
        'UUU':'동서바람성분','VEC':'풍향','VVV':'남북바람성분','WSD':'풍속'
       }, inplace=True) # 열의 이름을 해석

    redefined_columns = ['DATE_TIME','NX', 'NY', '기온', '시간1_강수량', '강수형태',
                         '습도', '풍속', '풍향', '동서바람성분', '남북바람성분'] # 열의 순서를 재조정 (펜시검색문법 활용)
    p_df = p_df[redefined_columns]
    return p_df

# 현재 시간 설정
base_date = time.strftime("%Y%m%d")
# print(base_date)
day_time = time.strftime("%H%M")
# print(day_time)

raw_str_json = get_request_url()
# print(raw_str_json)

if raw_str_json:
    raw_json = json.loads(raw_str_json)

# print(raw_json)

parsed_json = raw_json['response']['body']['items']['item']

# print(parsed_json)
df = pd.DataFrame(parsed_json)

df_preprocessed =  preprocess_df(df)
df_preprocessed

KeyError: 'body'

In [38]:
df_preprocessed.loc[:,'DATE_TIME'].iloc[0]

NameError: name 'df_preprocessed' is not defined

In [39]:
df_preprocessed.columns

NameError: name 'df_preprocessed' is not defined

In [40]:
insert_columns = ','.join(['DATE_TIME', 'NX', 'NY', '기온', '시간1_강수량', '강수형태', '습도', '풍속', '풍향',
       '동서바람성분', '남북바람성분'])
insert_columns

'DATE_TIME,NX,NY,기온,시간1_강수량,강수형태,습도,풍속,풍향,동서바람성분,남북바람성분'

In [41]:
insert_columns = ','.join(df_preprocessed.columns)
insert_columns

NameError: name 'df_preprocessed' is not defined

In [42]:
df_preprocessed.values

NameError: name 'df_preprocessed' is not defined

In [43]:
df_preprocessed.info()

NameError: name 'df_preprocessed' is not defined